In [1]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [2]:
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_montreal')

In [3]:
from src.tools.DD_insertion import construct_udd_sequence, \
                                 kdd_sequences, \
                                 kdd_spacing, \
                                 construct_bv_circuit, \
                                 construct_hs_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [7]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [8]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
629506380212ffac04251d6b
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295063af33cdb94432e04e7
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295063d4ad6555e8b9e48c4
if applied DD for None: False
if applied DD fo

In [9]:
for job_id in bv_job_ids:
    print(job_id)

629506380212ffac04251d6b
6295063af33cdb94432e04e7
6295063d4ad6555e8b9e48c4
629506409baeab7e8b4acb9d
6295064332e1768dde93c813
629506464bb9759416efb2e0
62950649f33cdb5e192e04e8
6295064c1ea57f5ffbebd475
6295065016cdc47b21f25c6d
6295065432e176a50693c814
6295065732e176df3c93c815
6295065a0212ffabda251d6c


In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '629506380212ffac04251d6b',
    '6295063af33cdb94432e04e7',
    '6295063d4ad6555e8b9e48c4',
    '629506409baeab7e8b4acb9d',
    '6295064332e1768dde93c813',
    '629506464bb9759416efb2e0',
    '62950649f33cdb5e192e04e8',
    '6295064c1ea57f5ffbebd475',
    '6295065016cdc47b21f25c6d',
    '6295065432e176a50693c814',
    '6295065732e176df3c93c815',
    '6295065a0212ffabda251d6c',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        obtain_results = result_count.get(ideal_result, 0)
        pst.append(obtain_results/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.4808349609375
pst result of hahn_X: 0.47998046875
pst result of hahn_Y: 0.48291015625
pst result of CP: 0.4886474609375
pst result of CPMG: 0.4794921875
pst result of xy4: 0.481201171875
pst result of xy8: 0.4354248046875
pst result of xy16: 0.4818115234375
pst result of udd1: 0.47021484375
pst result of udd2: 0.4554443359375
pst result of kdd: 0.4820556640625
---------Result of circuit BV4 -------------
pst result of no_dd: 0.4058837890625
pst result of hahn_X: 0.411865234375
pst result of hahn_Y: 0.416015625
pst result of CP: 0.418701171875
pst result of CPMG: 0.4139404296875
pst result of xy4: 0.419189453125
pst result of xy8: 0.4036865234375
pst result of xy16: 0.405517578125
pst result of udd1: 0.4066162109375
pst result of udd2: 0.4056396484375
pst result of kdd: 0.413818359375
---------Result of circuit BV5 -------------
pst result of no_dd: 0.0655517578125
pst result of hahn_X: 0.06640625
pst result of hahn_Y: 

In [8]:
bv_results_data

{'BV3': [0.4808349609375,
  0.47998046875,
  0.48291015625,
  0.4886474609375,
  0.4794921875,
  0.481201171875,
  0.4354248046875,
  0.4818115234375,
  0.47021484375,
  0.4554443359375,
  0.4820556640625],
 'BV4': [0.4058837890625,
  0.411865234375,
  0.416015625,
  0.418701171875,
  0.4139404296875,
  0.419189453125,
  0.4036865234375,
  0.405517578125,
  0.4066162109375,
  0.4056396484375,
  0.413818359375],
 'BV5': [0.0655517578125,
  0.06640625,
  0.0626220703125,
  0.32275390625,
  0.3175048828125,
  0.3280029296875,
  0.316650390625,
  0.3092041015625,
  0.318603515625,
  0.3009033203125,
  0.307861328125],
 'BV6': [0.227294921875,
  0.2379150390625,
  0.2392578125,
  0.29443359375,
  0.2718505859375,
  0.28857421875,
  0.3109130859375,
  0.28466796875,
  0.2911376953125,
  0.2691650390625,
  0.2603759765625],
 'BV7': [0.1181640625,
  0.1171875,
  0.1163330078125,
  0.2598876953125,
  0.240966796875,
  0.2474365234375,
  0.251220703125,
  0.2353515625,
  0.24365234375,
  0.23620

In [9]:

# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i / result_data[0] for i in result_data[1:]]
#     data.append(ratio)

# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[1:, 2]
# cpmg = data[1:, 3]
# xy4 = data[1:, 4]
# xy8 = data[1:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# udd_X = data[1:, 7]
# udd_Y = data[1:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', ]

# x_ticks = circuit_names[:6]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure

# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0, 0, 1, 1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:6], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:6], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:6], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:6], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:6], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:6], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:6], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:6], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)

In [10]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295065cf33cdbc8752e04e9
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295065e0212ff096b251d6d
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295065f1ea57f63dfebd476
if applied DD for None: False
if applied DD for N

In [11]:
for job_id in gs_job_ids:
    print(job_id)

6295065cf33cdbc8752e04e9
6295065e0212ff096b251d6d
6295065f1ea57f63dfebd476
6295066132e176096e93c816
629506629baeab7c124acb9e
629506643bb4af066bb95333
629506650212ffa6ca251d6e
629506671ea57f9fafebd477
629506694bb975df2cefb2e1
6295066b9baeab36b04acb9f
6295066d3bb4af6e96b95334
6295066f4bb97502e7efb2e2


In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '6295065cf33cdbc8752e04e9',
    '6295065e0212ff096b251d6d',
    '6295065f1ea57f63dfebd476',
    '6295066132e176096e93c816',
    '629506629baeab7c124acb9e',
    '629506643bb4af066bb95333',
    '629506650212ffa6ca251d6e',
    '629506671ea57f9fafebd477',
    '629506694bb975df2cefb2e1',
    '6295066b9baeab36b04acb9f',
    '6295066d3bb4af6e96b95334',
    '6295066f4bb97502e7efb2e2',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.056807027216801725
jsd result of hahn_X: 0.057652805417399096
jsd result of hahn_Y: 0.05351691088276701
jsd result of CP: 0.07900455584364804
jsd result of CPMG: 0.02531503904574524
jsd result of xy4: 0.04273867061632946
jsd result of xy8: 0.06441255188005343
jsd result of xy16: 0.05654965529466443
jsd result of udd1: 0.06308505021638162
jsd result of udd2: 0.03915772352534215
jsd result of kdd: 0.05351848205900959
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.04226548898966945
jsd result of hahn_X: 0.03806886939996354
jsd result of hahn_Y: 0.03873000522925532
jsd result of CP: 0.04059573745368716
jsd result of CPMG: 0.05581465756292323
jsd result of xy4: 0.05317151167744684
jsd result of xy8: 0.03866748134329414
jsd result of xy16: 0.06007792212817788
jsd result of udd1: 0.05807429879545776
jsd result of udd2: 0.032398667872253956
jsd result of kdd: 0.028550165052626628
---------Result of circuit

In [11]:
gs_results_data

{'GS3': [0.056807027216801725,
  0.057652805417399096,
  0.05351691088276701,
  0.07900455584364804,
  0.02531503904574524,
  0.04273867061632946,
  0.06441255188005343,
  0.05654965529466443,
  0.06308505021638162,
  0.03915772352534215,
  0.05351848205900959],
 'GS4': [0.04226548898966945,
  0.03806886939996354,
  0.03873000522925532,
  0.04059573745368716,
  0.05581465756292323,
  0.05317151167744684,
  0.03866748134329414,
  0.06007792212817788,
  0.05807429879545776,
  0.032398667872253956,
  0.028550165052626628],
 'GS5': [0.056576918228111533,
  0.059456961873251064,
  0.05717676665640321,
  0.05914385925276645,
  0.07225127830486094,
  0.07135633757119919,
  0.0683953472398619,
  0.08545999499730636,
  0.05494062563578569,
  0.05762755225929925,
  0.07514839612250791],
 'GS6': [0.06393339682987154,
  0.06466892287477435,
  0.05780176001743404,
  0.061711790705100966,
  0.05344325898176566,
  0.06283857538498129,
  0.06090542458351363,
  0.06313863626869594,
  0.0582821370616155

In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
629506724bb975cf1befb2e3
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
62950674bc0d022e86dbeb1d
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
629506771ea57ff933ebd478
if applied DD for None: False
if applied DD for N

In [13]:
for job_id in qft_job_ids:
    print(job_id)

629506724bb975cf1befb2e3
62950674bc0d022e86dbeb1d
629506771ea57ff933ebd478
6295067c4ad6555f909e48c6
629506804bb97555a1efb2e4
629506841ea57f9a73ebd479
6295068a16cdc45ae0f25c71
629506913bb4af14b3b95335
62950698f33cdbf5152e04ea
629506a21ea57f07fdebd47b
629506b43bb4afd130b95338
629506c10212ff76ff251d72


In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '629506724bb975cf1befb2e3',
    '62950674bc0d022e86dbeb1d',
    '629506771ea57ff933ebd478',
    '6295067c4ad6555f909e48c6',
    '629506804bb97555a1efb2e4',
    '629506841ea57f9a73ebd479',
    '6295068a16cdc45ae0f25c71',
    '629506913bb4af14b3b95335',
    '62950698f33cdbf5152e04ea',
    '629506a21ea57f07fdebd47b',
    '629506b43bb4afd130b95338',
    '629506c10212ff76ff251d72'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.016011501520206747
jsd result of hahn_X: 0.01812547624560923
jsd result of hahn_Y: 0.013234641005470182
jsd result of CP: 0.018554616864541632
jsd result of CPMG: 0.01963912850780635
jsd result of xy4: 0.03079028108960967
jsd result of xy8: 0.022859520918254463
jsd result of xy16: 0.028602521649041715
jsd result of udd1: 0.023155754270874154
jsd result of udd2: 0.024175158031272667
jsd result of kdd: 0.030312080997831687
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.09151372954646053
jsd result of hahn_X: 0.09957328055725853
jsd result of hahn_Y: 0.09768972716788253
jsd result of CP: 0.08871007796660277
jsd result of CPMG: 0.12111771688631216
jsd result of xy4: 0.12092991105757078
jsd result of xy8: 0.06922379929820897
jsd result of xy16: 0.11608539504384152
jsd result of udd1: 0.09956736563288207
jsd result of udd2: 0.11950360217032592
jsd result of kdd: 0.14770940636745045
---------Result of c

In [14]:
qft_results_data

{'QFT3': [0.016011501520206747,
  0.01812547624560923,
  0.013234641005470182,
  0.018554616864541632,
  0.01963912850780635,
  0.03079028108960967,
  0.022859520918254463,
  0.028602521649041715,
  0.023155754270874154,
  0.024175158031272667,
  0.030312080997831687],
 'QFT4': [0.09151372954646053,
  0.09957328055725853,
  0.09768972716788253,
  0.08871007796660277,
  0.12111771688631216,
  0.12092991105757078,
  0.06922379929820897,
  0.11608539504384152,
  0.09956736563288207,
  0.11950360217032592,
  0.14770940636745045],
 'QFT5': [0.12380557790711026,
  0.11719623482844709,
  0.13096977328116086,
  0.1198173737835448,
  0.10838290711048441,
  0.09600770050674697,
  0.0908796120261482,
  0.10412895535985499,
  0.10211849306119343,
  0.1101994280206466,
  0.09814269597225789],
 'QFT6': [0.14463574122749986,
  0.1381953605364347,
  0.1405113654544311,
  0.13579446363327552,
  0.10021155887062982,
  0.08712988275715607,
  0.09190480474076347,
  0.10371899045341196,
  0.142750002620231

In [15]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)